# --- Day 14: Extended Polymerization ---
## Part 1
The incredible pressures at this depth are starting to put a strain on your submarine. The submarine has polymerization equipment that would produce suitable materials to reinforce the submarine, and the nearby volcanically-active caves should even have the necessary input elements in sufficient quantities.

The submarine manual contains instructions for finding the optimal polymer formula; specifically, it offers a polymer template and a list of pair insertion rules (your puzzle input). You just need to work out what polymer would result after repeating the pair insertion process a few times.

For example:
```
NNCB

CH -> B
HH -> N
CB -> H
NH -> C
HB -> C
HC -> B
HN -> C
NN -> C
BH -> H
NC -> B
NB -> B
BN -> B
BB -> N
BC -> B
CC -> N
CN -> C
```
The first line is the polymer template - this is the starting point of the process.

The following section defines the pair insertion rules. A rule like AB -> C means that when elements A and B are immediately adjacent, element C should be inserted between them. These insertions all happen simultaneously.

So, starting with the polymer template NNCB, the first step simultaneously considers all three pairs:

    The first pair (NN) matches the rule NN -> C, so element C is inserted between the first N and the second N.
    The second pair (NC) matches the rule NC -> B, so element B is inserted between the N and the C.
    The third pair (CB) matches the rule CB -> H, so element H is inserted between the C and the B.

Note that these pairs overlap: the second element of one pair is the first element of the next pair. Also, because all pairs are considered simultaneously, inserted elements are not considered to be part of a pair until the next step.

After the first step of this process, the polymer becomes NCNBCHB.

Here are the results of a few steps using the above rules:
```
Template:     NNCB
After step 1: NCNBCHB
After step 2: NBCCNBBBCBHCB
After step 3: NBBBCNCCNBBNBNBBCHBHHBCHB
After step 4: NBBNBNBBCCNBCNCCNBBNBBNBBBNBBNBBCBHCBHHNHCBBCBHCB
```
This polymer grows quickly. After step 5, it has length 97; After step 10, it has length 3073. After step 10, B occurs 1749 times, C occurs 298 times, H occurs 161 times, and N occurs 865 times; taking the quantity of the most common element (B, 1749) and subtracting the quantity of the least common element (H, 161) produces 1749 - 161 = 1588.

Apply 10 steps of pair insertion to the polymer template and find the most and least common elements in the result. What do you get if you take the quantity of the most common element and subtract the quantity of the least common element?


In [1]:
from collections import defaultdict
import copy as cp
import pandas as pd
from tqdm import tqdm
from typing import DefaultDict

In [2]:
polymer_formula: DefaultDict[str, str] = defaultdict(lambda: "")
with open("../data/day14.dat", "r") as IH:
    polymer_template = next(IH).strip()
    next(IH)
    for line in IH:
        dimer, _, output = line.strip().split(" ")
        polymer_formula[dimer] = output

In [3]:
def apply_polymer_formula(polymer: str, formula: DefaultDict[str, str]) -> str:
    output_polymer = ""
    for did in range(len(polymer)-1):
        output_polymer += polymer[did] + formula[polymer[did:did+2]]
    return output_polymer + polymer[-1]

In [4]:
print(f"Template:\t{polymer_template}")

Template:	SHHNCOPHONHFBVNKCFFC


In [5]:
# Iteration approach
polymer = cp.copy(polymer_template)
for step_id in tqdm(range(10), desc="step"):
    polymer = apply_polymer_formula(polymer, polymer_formula)

step: 100%|██████████| 10/10 [00:00<00:00, 1003.93it/s]


In [6]:
monomer_counts: DefaultDict[str, int] = defaultdict(lambda: 0)
for monomer in polymer:
    monomer_counts[monomer] += 1

In [7]:
monomers = pd.DataFrame(monomer_counts, index=pd.Index([0])).transpose()
monomers.columns = ["count"]
monomers.sort_values("count", inplace=True)

In [8]:
monomers["count"].values[-1]-monomers["count"].values[0]

2549

## Minimal approach with dimer counting

In [9]:
def polymer_to_dimer_count(polymer: str) -> DefaultDict[str, int]:
    polymer_dimer_count: DefaultDict[str, int] = defaultdict(lambda: 0)
    for i in range(len(polymer)):
        polymer_dimer_count[polymer[i:i+2]] += 1
    return polymer_dimer_count

In [10]:
def apply_step_to_dimer_count(dimer_count: DefaultDict[str, int], formula: DefaultDict[str, str]) -> DefaultDict[str, int]:
    to_add: DefaultDict[str, str] = defaultdict(lambda: 0)
    for dimer, nt_insert in formula.items():
        if dimer in dimer_count:
            to_add[dimer[0] + nt_insert] += dimer_count[dimer]
            to_add[nt_insert + dimer[1]] += dimer_count[dimer]
            dimer_count[dimer] = 0
    for dimer, count in to_add.items():
        dimer_count[dimer] += count
    return dimer_count

In [11]:
def dimer_to_monomer_count(dimer_count: DefaultDict[str, int]) -> DefaultDict[str, int]:
    monomer_count: DefaultDict[str, int] = defaultdict(lambda: 0)
    for dimer, count in dimer_count.items():
        monomer_count[dimer[0]] += count
    return monomer_count

In [12]:
dimer_count = polymer_to_dimer_count(polymer_template)

In [13]:
for step_id in tqdm(range(10), desc="step"):
    dimer_count = apply_step_to_dimer_count(dimer_count, polymer_formula)

step: 100%|██████████| 10/10 [00:00<00:00, 3789.23it/s]


In [14]:
monomer_count = dimer_to_monomer_count(dimer_count)
sorted_monomer_count = sorted(list(monomer_count.items()), key=lambda x: x[1])
sorted_monomer_count[-1][1] - sorted_monomer_count[0][1]

2549

## --- Part Two ---

The resulting polymer isn't nearly strong enough to reinforce the submarine. You'll need to run more steps of the pair insertion process; a total of 40 steps should do it.

In the above example, the most common element is B (occurring 2192039569602 times) and the least common element is H (occurring 3849876073 times); subtracting these produces 2188189693529.

Apply 40 steps of pair insertion to the polymer template and find the most and least common elements in the result. What do you get if you take the quantity of the most common element and subtract the quantity of the least common element?


In [15]:
dimer_count = polymer_to_dimer_count(polymer_template)

In [16]:
for step_id in tqdm(range(40), desc="step"):
    dimer_count = apply_step_to_dimer_count(dimer_count, polymer_formula)

step: 100%|██████████| 40/40 [00:00<00:00, 10313.02it/s]


In [17]:
monomer_count = dimer_to_monomer_count(dimer_count)
sorted_monomer_count = sorted(list(monomer_count.items()), key=lambda x: x[1])
sorted_monomer_count[-1][1] - sorted_monomer_count[0][1]

2516901104210